In [1]:
## Left Turn/Right Turn Intersections(2013-2017)

#### Database: DOTDEVRHPGSQL01
#### Tables: archive."2019_11_13_nysdot_all", archive."2019_11_13_nysdot_vehicle"

### 1.Read in xl
### 2.NYSDOT Check if column AE =Yes then get all bike or ped injuries at that intersection
###   where the (non-bike) vehicle was making a left turn 2013-2017
###   Add the count of included crashes into the excel 

### 3.NYSDOT Check if column AF =Yes then get all bike or ped injuries at that intersection 
###   where the (non-bike) vehicle was making a right turn 2013-2017
###   Add the count of included crashes into the excel 
### 
### 4.Get the mv104s for each intersection with 2 or more injuries.


In [2]:
from ris import pysqldb
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

print 'Notebook run: {}'.format(timestamp)
print os.getcwd()

Notebook run: 2019-12-10_17:15
E:\RIS\Staff Folders\Samuel\Requests\SH\LT Injury Itx


In [3]:
#Database connection
cdb = pysqldb.DbConnect(server='DOTDEVRHPGSQL01', database='CRASHDATA', type='PG')


Additional database connection details required:
User name (crashdata):soge
Password (crashdata)········
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2019-12-10 17:15:07


In [4]:
# Reading in xl

ltrt = pd.read_excel('E:\RIS\Staff Folders\Samuel\Requests\SH\LT Injury Itx\Feasible Locations to Review Crashes.xlsx')

In [5]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2019-12-10 17:15:11


In [6]:
#Writing filtered table column names to pgsql

cdb.dataframe_to_table_schema(ltrt, 'LTRT', schema='working', overwrite = True)

[['street', 'varchar (500)'],
 ['number_tra', 'bigint'],
 ['status', 'varchar (500)'],
 ['boro', 'bigint'],
 ['nodeid', 'bigint'],
 ['masterid', 'bigint'],
 ['signal_type', 'varchar (500)'],
 ['lat', 'float'],
 ['long', 'float'],
 ['street1', 'varchar (500)'],
 ['street2', 'varchar (500)'],
 ['max_vph', 'float'],
 ['coverage', 'float'],
 ['coverage_half_mile', 'varchar (500)'],
 ['request_count', 'bigint'],
 ['request_binary', 'varchar (500)'],
 ['16-18_ped_injuries', 'bigint'],
 ['16-18_injuries', 'bigint'],
 ['16-18_crashes', 'bigint'],
 ['18_ped_injuries', 'bigint'],
 ['18_injuries', 'bigint'],
 ['18_crashes', 'bigint'],
 ['legs', 'bigint'],
 ['preliminary_geometric_lttc/rttc_feasibility', 'varchar (500)'],
 ['lttc/rttc_feasible_leg', 'varchar (500)'],
 ['reason_if_not_feasible', 'varchar (500)'],
 ['coordination_notes', 'varchar (500)'],
 ['multilane_(yes/no)', 'varchar (500)'],
 ['appr_that_is_multilane', 'varchar (500)'],
 ['appr_that_is_singlelane', 'varchar (500)'],
 ['provide_

In [7]:
#Writing actual data from filtered table to created table in pgsql


input_schema = cdb.dataframe_to_table_schema(ltrt, 'LTRT', schema='working', overwrite = True)
cdb.dataframe_to_table(ltrt, 'LTRT', input_schema, schema= "working")

Reading data into Database



152it [00:03, 44.03it/s]



152 rows added to working.LTRT



In [8]:
#Here, the number of valid injuries per intersection is selected. 
#Valid meaning if the request was for all left turn injuries 
#at a specific interection, only left turn injuries are counted at that node


result = cdb.dfquery("""    

with data as(

--Here, we select all crashes and the number of injuries between the years 2013 through 2017 where the vehicle 
--was either making a left turn or a right turn and hit a bicyclist or pedestrian. All nodeids that are not 
--selected in the working."LTRT" table

select distinct 
    nodeid, nys_a.crashid, nys_a.case_num, nys_a.case_yr, num_of_inj, accd_type_int, pre_accd_actn 
from archive."2019_11_13_nysdot_all" nys_a
join archive."2019_11_13_nysdot_vehicle" nys_v 
    on nys_a.crashid = nys_v.crashid
where accd_type_int in (1,2) -- ped/bike collision
    and nodeid::bigint in (SELECT nodeid::bigint FROM working."LTRT")
    and nys_v.pre_accd_actn in ('02', '03') -- 02 making a right turn, 03 making a left turn
    and nys_a.case_yr>= 2013 and nys_a.case_yr<=2017
    and nys_v.veh_typ::int not in (5,6) -- vehicle type ped/bike
) 

--This query sums up the total number of valid injuries for every node 
    select nodeid, rt, lt, (rt_inj + lt_inj + ltrt_inj) tot_inj  
    from(
        -- This subquery sums up all the valid injuries and groups them by nodeid.
        select nodeid, rt, lt,  

            -- sum of all the right turn injuries where the request at the intersection was for right turns
            coalesce(sum(case when (rt = 'Yes' and lt = 'No' and pre_accd_actn::text = '02') then num_of_inj end),0) rt_inj,

            -- sum of all the left turn injuries where the request at the intersection was for left turns
            coalesce(sum(case when (rt = 'No' and lt = 'Yes' and pre_accd_actn::text = '03') then num_of_inj end),0) lt_inj,

            -- sum of all the left turn and right turn injuries where the request at the intersection was for both left and right turns 
            coalesce(sum(case when (rt = 'Yes' and lt = 'Yes') then num_of_inj end),0) ltrt_inj 
            from(
                -- This subquery selects all the crashes for the nodes in the working."LTRT" table and 
                -- incicates the right turn/left turn request

                select data.nodeid, num_of_inj,pre_accd_actn,
                    "Provide RT Ped & Bike Injuries" RT,
                    "Provide LT Ped & Bike Injuries" LT 
                from data 
                join working."LTRT" ltrt
                    on data.nodeid::int = ltrt.nodeid::int
        ) valid_injuries 
        group by nodeid, rt, lt
    )total_injuries
    """)

In [9]:
#Mapping nodeids from query to integer in order to join injury counts to original excel sheet

result['nodeid'] = map(int, result['nodeid'])

In [10]:
#Merging total injurry count by node to original excel sheet that was read in. 
#After merge, all null values are filled with zeros. 

ltrt = ltrt.merge(result[['nodeid','tot_inj']], how='left', on='nodeid')
ltrt = ltrt.fillna(0)

In [11]:
#Writing original table with total injury counts to csv. 

ltrt.to_csv('LT_RT_injuries.csv')

In [12]:
#Here, we only select the intersections where there were two or more 
#valid injuries.


cases = cdb.dfquery("""    


with data as(

--Here, we select all crashes and the number of injuries between the years 2013 through 2017 where the vehicle 
--was either making a left turn or a right turn and hit a bicyclist or pedestrian. All nodeids that are not 
--selected in the working."LTRT" table

select distinct 
    nodeid, nys_a.crashid, nys_a.case_num, nys_a.case_yr, num_of_inj, accd_type_int, pre_accd_actn 
from archive."2019_11_13_nysdot_all" nys_a
join archive."2019_11_13_nysdot_vehicle" nys_v 
    on nys_a.crashid = nys_v.crashid
where accd_type_int in (1,2) -- ped/bike collision
    and nodeid::bigint in (SELECT nodeid::bigint FROM working."LTRT")
    and nys_v.pre_accd_actn in ('02', '03') -- 02 making a right turn, 03 making a left turn
    and nys_a.case_yr>= 2013 and nys_a.case_yr<=2017
    and nys_v.veh_typ::int not in (5,6) -- vehicle type ped/bike
) 


--Here, we select all crashes and the number of injuries between the years 2013 through 2017 where the vehicle 
--was either making a left turn or a right turn and hit a bicyclist or pedestrian. 

select data.nodeid, (ltrt2.street1, ltrt2.street2) as streets, data.case_num, data.case_yr  
from( 
    --This query sums up the total number of valid injuries for every node 
    select nodeid, rt, lt, (rt_inj + lt_inj + ltrt_inj) tot_inj  
    from(
        -- This subquery sums up all the valid injuries and groups them by nodeid.
        select nodeid, rt, lt,  

            -- sum of all the right turn injuries where the request at the intersection was for right turns
            coalesce(sum(case when (rt = 'Yes' and lt = 'No' and pre_accd_actn::text = '02') then num_of_inj end),0) rt_inj,

            -- sum of all the left turn injuries where the request at the intersection was for left turns
            coalesce(sum(case when (rt = 'No' and lt = 'Yes' and pre_accd_actn::text = '03') then num_of_inj end),0) lt_inj,

            -- sum of all the left turn and right turn injuries where the request at the intersection was for both left and right turns 
            coalesce(sum(case when (rt = 'Yes' and lt = 'Yes') then num_of_inj end),0) ltrt_inj 
            from(
                -- This subquery selects all the crashes for the nodes in the working."LTRT" table and 
                -- incicates the right turn/left turn request

                select data.nodeid, num_of_inj,pre_accd_actn,
                    "Provide RT Ped & Bike Injuries" RT,
                    "Provide LT Ped & Bike Injuries" LT 
                from data 
                join working."LTRT" ltrt
                    on data.nodeid::int = ltrt.nodeid::int
        ) valid_injuries 
                group by nodeid, rt, lt
    )total_injuries 
)ltrt_injuries
left join data on -- joining back to data to retrieve the case_numbers and case years for each case.
ltrt_injuries.nodeid = data.nodeid
left join working."LTRT" ltrt2 -- joining with working."LTRT" table once more to grab the street names for each resultant node
on ltrt_injuries.nodeid::int = ltrt2.nodeid::int
where tot_inj >=2 -- selecting locations where total number of injuries is greater or equal to 2


""")

In [13]:
#Writing table containing:
#*nodeid, streets, case_num, case_yr* for all intersections 
#with 2 or more valid injuries to csv.

cases.to_csv('nodes_w_mv104.csv', index = False)

In [14]:
for_mv104 = cases[['case_num','case_yr']]

In [15]:
#writing case numbers and case years to csv for getting mv104s

for_mv104.to_csv('cases_for_mv104.csv', index = False)